In [1]:
import polars as pl
import sys
import yaml
import os
import numpy as np
import pandas as pd

In [2]:
sys.path.append('../src/')
project_root = os.path.dirname(os.getcwd())

In [3]:
# Directorios para los archivos de parámetros y los datos
parameters_directory = os.path.join(project_root, 'src', 'parameters')
data_raw_directory = os.path.join(project_root, 'data', 'raw')
data_processed_directory = os.path.join(project_root, 'data', 'processed')
data_features_directory = os.path.join(project_root, 'data', 'featured')
data_train_directory = os.path.join(project_root, 'data', 'model_input', 'train')

In [4]:
# Lista todos los archivos YAML en el directorio especificado
yaml_files = [f for f in os.listdir(parameters_directory) if f.endswith('.yml')]

# Diccionario para guardar los parámetros cargados
parameters = {}

# Carga cada archivo YAML
for yaml_file in yaml_files:
    with open(os.path.join(parameters_directory, yaml_file), 'r') as file:
        data = yaml.safe_load(file)
        key_name = f'parameters_{yaml_file.replace(".yml", "")}'
        parameters[key_name] = data

PROCESSING DATA

In [5]:
# # Nodos
# from functions.processing import (validate_tags_pl, 
#                              validate_dtypes_pl, 
#                              change_names_pl, 
#                              change_dtype_pl,
#                              delete_accents_pl,
#                              standardize_binary_values_pl,
#                              impute_missing_values_pl)

In [6]:
# tag_dict_path = os.path.join(data_raw_directory, parameters['parameters_catalog']['tag_dict_path'])
# raw_data_path = os.path.join(data_raw_directory, parameters['parameters_catalog']['raw_data_path'])
# 
# tag_dict = pl.read_excel(tag_dict_path)
# data_raw = pl.read_csv(raw_data_path)

In [7]:
# prueba1 = validate_tags_pl(data_raw, tag_dict)

In [8]:
# prueba2 = validate_dtypes_pl(prueba1, tag_dict)

In [9]:
# prueba3 = change_names_pl(prueba2, tag_dict)

In [10]:
# prueba4 = change_dtype_pl(prueba3, tag_dict)

In [11]:
# prueba5 = delete_accents_pl(prueba4)

In [12]:
# prueba6 = standardize_binary_values_pl(prueba5, parameters['parameters_processing'])

In [13]:
# prueba7 = impute_missing_values_pl(prueba6)

FEATURING DATA

In [14]:
# # Nodos
# from functions.featuring import (new_features_pl,
#                                  add_target_variable_pl,
#                                  one_hot_encoding_pl,
#                                  random_forest_selection_pl,
#                                  conditional_entropy_selection_pl,
#                                  intersect_top_features_pl)`

In [15]:
# processed_data_path = os.path.join(data_processed_directory, parameters['parameters_catalog']['processed_data_path'])
#
# data_processed = pl.read_csv(processed_data_path)

In [16]:
# prueba8 = new_features_pl(data_processed, parameters['parameters_featuring'])

In [17]:
# target_directory = os.path.join(project_root, 'data', 'processed')
# target_path = os.path.join(target_directory, parameters['parameters_catalog']['target_column_path'])
# target = pl.read_csv(target_path)
# prueba9 = add_target_variable_pl(prueba8, target, parameters['parameters_featuring'])

In [18]:
# prueba10 = one_hot_encoding_pl(prueba9, parameters['parameters_featuring'])

In [19]:
# prueba11 = random_forest_selection_pl(prueba10, parameters['parameters_featuring'])

In [20]:
# prueba12 = conditional_entropy_selection_pl(prueba10, parameters['parameters_featuring'])

In [21]:
# prueba13 = intersect_top_features_pl(prueba11, prueba12, prueba10, parameters['parameters_featuring'])

MODEL INPUT DATA

In [22]:
# # Nodos
# from functions.model_input import (min_max_scaler_pl,
#                                    balance_target_variable_pl,
#                                    train_test_split_pl)

In [23]:
# featured_data_path = os.path.join(data_features_directory, parameters['parameters_catalog']['features_data_path'])
#
# data_featured = pl.read_csv(featured_data_path)

In [24]:
# prueba14 = min_max_scaler_pl(data_featured)

In [25]:
# prueba15 = balance_target_variable_pl(prueba14, parameters['parameters_model_input'])

In [26]:
# prueba16 = train_test_split_pl(prueba15, parameters['parameters_model_input'])

MODELS

In [27]:
# Nodos
from functions.models import (train_basic_models_pd)

In [28]:
train_data_path = os.path.join(data_train_directory, parameters['parameters_catalog']['train_data_path'])

train_data = pd.read_csv(train_data_path)

In [29]:
prueba17 = train_basic_models_pd(train_data, parameters['parameters_models'])

INFO:functions.models:Inicio del entrenamiento y evaluación de modelos básicos...
INFO:functions.models:Realizando análisis estadístico de comparación
INFO:functions.models:Estadístico de Friedman: 90.0, P-value: 2.190570119285308e-19


In [30]:
prueba17

{'Logistic Regression': {'accuracy': array([0.76863636, 0.74727273, 0.75954545, 0.76136364, 0.76318182,
         0.76216462, 0.77808095, 0.75807185, 0.76989541, 0.77080491,
         0.77818182, 0.75136364, 0.76772727, 0.76136364, 0.75545455,
         0.75989086, 0.77216917, 0.76307412, 0.76216462, 0.76716689,
         0.75818182, 0.76090909, 0.75136364, 0.76363636, 0.76909091,
         0.77717144, 0.77444293, 0.75898136, 0.75807185, 0.76989541]),
  'precision': array([0.77086495, 0.72056738, 0.74780059, 0.77056962, 0.74714286,
         0.75945537, 0.7795858 , 0.74063401, 0.76162791, 0.7630814 ,
         0.7898773 , 0.73255814, 0.76888218, 0.75295858, 0.73944687,
         0.75299401, 0.76410999, 0.75862069, 0.74053296, 0.75953079,
         0.74344023, 0.75872534, 0.74539877, 0.75703704, 0.77321157,
         0.77893175, 0.7696793 , 0.73529412, 0.73721591, 0.76946108]),
  'recall': array([0.58660508, 0.58592849, 0.58823529, 0.56170704, 0.60322953,
         0.57967667, 0.60854503, 0.593533